# Train a neural network (gaussian noise during training and predicting)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Set seeds

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

Set parameters

In [ ]:
# file with training input and output data: format is x,y
input_file = 'training.csv'

# names of input/output columns
inputs = ['mu', 'angle', 'threshold']
outputs = ['low_speed', 'high_speed']

# these set the input/output dimensions of the network
input_size = len(inputs)
output_size = len(outputs)

Read data

In [ ]:
data = pd.read_csv(input_file)
data.columns = inputs + outputs

x = np.array(data[inputs])
y = np.array(data[outputs])

Normalizing preprocessing layer from training data

In [ ]:
normalizer = keras.layers.experimental.preprocessing.Normalization()
normalizer.adapt(x)

Base model

In [ ]:
input_shape = x.shape[1:]
full_model_input = keras.Input(shape=input_shape)
normalized_input = normalizer(full_model_input)
noise_layer_1 = keras.layers.GaussianNoise(0.025)(normalized_input[...,0],training=True)
noise_layer_2 = keras.layers.GaussianNoise(0.025)(normalized_input[...,1],training=True)
noise_layer_3 = keras.layers.GaussianNoise(0)(normalized_input[...,2],training=True)
reshape_layer_1 = keras.layers.Reshape((1,))(noise_layer_1)
reshape_layer_2 = keras.layers.Reshape((1,))(noise_layer_2)
reshape_layer_3 = keras.layers.Reshape((1,))(noise_layer_3)
recombined_input = keras.layers.Concatenate()([reshape_layer_1, reshape_layer_2, reshape_layer_3])
dense_layer = keras.layers.Dense(20, activation='relu')(recombined_input)
full_model_output = keras.layers.Dense(output_size,activation='relu')(dense_layer)
full_model = keras.Model(full_model_input, full_model_output)
full_model.compile(loss='mse',optimizer='adam',metrics=['mse'])

Split data

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.33, shuffle= True)

Train model

In [ ]:
model_output = full_model.fit(x_train,y_train,epochs=600,batch_size=10,verbose=0,validation_data=(x_val,y_val)) # check validation

View training

In [ ]:
plt.yscale('log')
plt.title('loss')
plt.plot(model_output.history['loss'], label='train')
plt.plot(model_output.history['val_loss'], label='validation')
plt.legend()
plt.figure()
plt.yscale('log')
plt.title('mse')
plt.plot(model_output.history['mse'], label='train')
plt.plot(model_output.history['val_mse'], label='validation')
plt.legend()
plt.figure()

View output

In [ ]:
# plot speed vs angle given mu, threshold
mu = 0.009 # set mu
thresh = 4 # set threshold

# bug: mu = 0.009 is read as 0.0090..01
#plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
plot_x = [i for i in x if np.isclose(i[0], mu) and i[2] == thresh] # x, y
# this is not generic enough...
pred_x = [[mu,angle,thresh] for angle in np.linspace(0,165,165)]
pred = full_model.predict(pred_x)

plt.title('mu = %s, thresh = %s' % (mu, thresh))
plt.xlabel('angle (degrees)')
plt.ylabel('speed (m/s)')
#plt.plot([i[1] for i in plot_values], [y[i] for i,v in enumerate(x) if v[0] == mu and v[2] == thresh])
plt.plot([i[1] for i in plot_x], [y[i] for i,v in enumerate(x) if np.isclose(v[0], mu) and v[2] == thresh], marker='o')
plt.plot([i[1] for i in pred_x], pred, label='pred')
plt.legend()

In [ ]:
# plot speed vs angle given mu, threshold
mu = 0.009 # set mu
thresh = 4 # set threshold

# bug: mu = 0.009 is read as 0.0090..01
#plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
for i in range(0, 5):
    plot_x = [i for i in x if np.isclose(i[0], mu) and i[2] == thresh] # x, y
    # this is not generic enough...
    pred_x = [[mu,angle,thresh] for angle in np.linspace(0,165,165)]
    pred = full_model.predict(pred_x)
    plt.scatter([i[1] for i in pred_x], pred[...,1])
plt.plot([i[1] for i in plot_x], [y[i][1] for i,v in enumerate(x) if np.isclose(v[0], mu) and v[2] == thresh], label='train')

Save model

In [ ]:
full_model.save("flat_network_with_noise.h5")

Find largest divergence between prediction and training data

In [ ]:
# divergence = (full_model.predict(x) - y)
# max_divergence = max([i[1] for i in divergence])
# print(max_divergence)
# print(max_divergence, x[np.where(divergence == max_divergence)[0]])

Time the network

In [ ]:
# %%timeit time_x = np.array(np.random.rand(1,3))
# full_model.predict(time_x)

In [ ]:
# %%timeit time_x = np.array(np.random.rand(10,3))
# full_model.predict(time_x)

In [ ]:
# %%timeit time_x = np.array(np.random.rand(100,3))
# full_model.predict(time_x)

# Predictions

Live variance calculation and adjustment

In [ ]:
# plot speed vs angle given mu, threshold
mu = 0.009 # set mu
thresh = 4 # set threshold
runs = 5 # runs per prediction

# bug: mu = 0.009 is read as 0.0090..01
#plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
plot_x = [i for i in x if np.isclose(i[0], mu) and i[2] == thresh] # x, y
# this is not generic enough...
pred_x = [[mu,angle,thresh] for angle in np.linspace(0,165,165)]
pred = []

plt.title('mu = %s, thresh = %s' % (mu, thresh))
plt.xlabel('angle (degrees)')
plt.ylabel('speed (m/s)')
plt.plot([i[1] for i in plot_x], [y[i][1] for i,v in enumerate(x) if np.isclose(v[0], mu) and v[2] == thresh], marker='o', label='training')

for i in pred_x:
    point_predictions = full_model.predict([i for run in range(0, runs)])[...,1] # faster than np.tile for small arrays
    point_predictions = np.array(point_predictions)
    variance = point_predictions.var(axis = 0, ddof = 1) # unbiased estimator
    mean = point_predictions.mean()
    pred.append(mean - 2 * (variance**0.5))

plt.scatter([i[1] for i in pred_x], pred, label='predictions', c='orange')
plt.legend()

Timing

In [ ]:
runs = 100

In [ ]:
%%timeit time_x = [np.random.randn(1)[0], np.random.randn(1)[0], np.random.randn(1)[0]]
point_predictions = full_model.predict([time_x for run in range(0, runs)])[...,1] # faster than np.tile for small arrays
point_predictions = np.array(point_predictions)
variance = point_predictions.var(axis = 0, ddof = 1) # unbiased estimator
mean = point_predictions.mean()
predicted_speed = (mean - 2 * (variance**0.5))

Some code for offline predictions

In [ ]:
# # plot speed vs angle given mu, threshold with variance
# mu = 0.009 # set mu
# thresh = 4 # set threshold
# plot_deviation = standard_deviation*2 # 2 std

# # bug: mu = 0.009 is read as 0.0090..01
# plot_x = [i for i in x if np.isclose(i[0], mu) and i[2] == thresh] # x, y
# # this is not generic enough...
# pred_x = [[mu,angle,thresh] for angle in np.linspace(0,165,165)]

# plt.title('mu = %s, thresh = %s' % (mu, thresh))
# plt.xlabel('angle (degrees)')
# plt.ylabel('speed (m/s)')
# #plt.plot([i[1] for i in plot_values], [y[i] for i,v in enumerate(x) if v[0] == mu and v[2] == thresh])
# plt.plot([i[1] for i in plot_x], [y[i][1] for i,v in enumerate(x) if np.isclose(v[0], mu) and v[2] == thresh], marker='o')
# #plt.scatter([i[1] for i in pred_x], pred[...,1], label='pred')
# #plt.scatter([i[1] for i in pred_x], pred[...,1]-plot_deviation, label='pred - std')
# for i in range(0, 100):
#     pred = full_model.predict(pred_x)
#     plt.scatter([i[1] for i in pred_x], pred[...,1]-plot_deviation, c='orange', alpha=0.01)

In [ ]:
# subtract_value = tf.convert_to_tensor([standard_deviation*2]) # subtract 2 std from predictions

# weights = [layer.get_weights() for layer in full_model.layers]

# input_shape = x.shape[1:]
# prediction_model_input = keras.Input(shape=input_shape)
# normalized_input = normalizer(prediction_model_input)
# noise_layer_1 = keras.layers.GaussianNoise(0.025)(normalized_input[...,0],training=False)
# noise_layer_2 = keras.layers.GaussianNoise(0.025)(normalized_input[...,1],training=False)
# noise_layer_3 = keras.layers.GaussianNoise(0)(normalized_input[...,2],training=False)
# reshape_layer_1 = keras.layers.Reshape((1,))(noise_layer_1)
# reshape_layer_2 = keras.layers.Reshape((1,))(noise_layer_2)
# reshape_layer_3 = keras.layers.Reshape((1,))(noise_layer_3)
# recombined_input = keras.layers.Concatenate()([reshape_layer_1, reshape_layer_2, reshape_layer_3])
# dense_layer = keras.layers.Dense(20, activation='relu')(recombined_input)
# prediction_model_output = keras.layers.Dense(output_size,activation='relu')(dense_layer)
# prediction_model_subtract = keras.layers.Subtract()([prediction_model_output, subtract_value])
# prediction_model = keras.Model(prediction_model_input, prediction_model_subtract)
# prediction_model.compile(loss='mse',optimizer='adam',metrics=['mse'])

# for i in range(len(weights)):
#     prediction_model.layers[i].set_weights(weights[i])

In [ ]:
# # plot speed vs angle given mu, threshold
# mu = 0.009 # set mu
# thresh = 4 # set threshold

# # bug: mu = 0.009 is read as 0.0090..01
# #plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
# plot_x = [i for i in x if np.isclose(i[0], mu) and i[2] == thresh] # x, y
# # this is not generic enough...
# pred_x = [[mu,angle,thresh] for angle in np.linspace(0,165,165)]
# pred = prediction_model.predict(pred_x)

# plt.title('mu = %s, thresh = %s' % (mu, thresh))
# plt.xlabel('angle (degrees)')
# plt.ylabel('speed (m/s)')
# #plt.plot([i[1] for i in plot_values], [y[i] for i,v in enumerate(x) if v[0] == mu and v[2] == thresh])
# plt.plot([i[1] for i in plot_x], [y[i][1] for i,v in enumerate(x) if np.isclose(v[0], mu) and v[2] == thresh], marker='o')
# plt.plot([i[1] for i in pred_x], pred[...,1]+subtract_value, label='pred') # change this to an actual prediction
# plt.plot([i[1] for i in pred_x], pred[...,1], label='pred - 2std')
# plt.legend()

In [ ]:
# prediction_model.save("gaussian_prediction_network.h5")

Make a lot of plots

In [ ]:
# # only plot the high prediction...
# mus = data['mu'].unique()
# angles = data['angle'].unique()
# thresholds = data['threshold'].unique()
# a=0
# # plot speed vs angle given mu, threshold
# for mu in mus:
#     for threshold in thresholds:
#         # bug: mu = 0.009 is read as 0.0090..01
#         #plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
#         plot_x = [i for i in x if np.isclose(i[0], mu) and i[2] == threshold] # x, y
#         pred_x = [[mu,angle,threshold] for angle in np.linspace(angles.min(),angles.max(),angles.max())]
#         pred = prediction_model.predict(pred_x)
#         #plt.plot([i[1] for i in plot_values], [y[i] for i,v in enumerate(x) if v[0] == mu and v[2] == thresh])
#         fig = plt.figure()
#         plt.title('mu: %.3f, threshold: %.2f' % (mu, threshold))
#         plt.plot([i[1] for i in plot_x], [y[i][1] for i,v in enumerate(x) if np.isclose(v[0], mu) and v[2] == threshold],
#                 label = 'training')
#         plt.plot([i[1] for i in pred_x], [i[1] for i in pred], label = 'predicted')
#         plt.legend()
#         plt.savefig('plots/mu-%.3f_threshold-%.2f.png' % (mu,threshold))
#         plt.close()

Some plots for the paper

In [ ]:
# plot speed vs angle given mu, threshold
mu = 0.05 # set mu
#thresh = data['threshold'].unique() # set threshold
thresh = [1.0, 2.0, 3.0, 4.0]

# diff markers
m = 0
markers = ['o', 's', 'd', '*']

#plt.title('mu = %s' % (mu))#, thresh))
plt.xlabel('Angle (degrees)',size='18')
plt.ylabel('Speed (m/s)',size='18')
plt.ylim(0.9, 2.1)
for t in thresh:
    plot_x = [i for i in x if np.isclose(i[0], mu) and i[2] == t] # x, y
    plt.plot([i[1] for i in plot_x], [y[i][1] for i,v in enumerate(x) if np.isclose(v[0], mu) and v[2] == t], marker=markers[m], label='%d m' % (t))
    m+=1
plt.tight_layout()
plt.legend()
plt.savefig('0.05.png')

In [ ]:
# plot speed vs angle given mu, threshold
mu = 0.05 # set mu
thresh = 4 # set threshold
runs = 10 # runs per prediction

# bug: mu = 0.009 is read as 0.0090..01
#plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
plot_x = [i for i in x if np.isclose(i[0], mu) and i[2] == thresh] # x, y
# this is not generic enough...
pred_x = [[mu,angle,thresh] for angle in np.linspace(0,165,165)]
pred = []

plt.title('Uncertainty Adjustment at mu = %s, threshold = %s m' % (mu, thresh))
plt.xlabel('Angle (degrees)', size=18)
plt.ylabel('Speed (m/s)', size=18)
plt.plot([i[1] for i in plot_x], [y[i][1] for i,v in enumerate(x) if np.isclose(v[0], mu) and v[2] == thresh], marker='o', label='training')

# unadjusted predictions

for i in pred_x:
    point_predictions = full_model.predict([i for run in range(0, 1)])[...,1] # faster than np.tile for small arrays
    point_predictions = np.array(point_predictions)
    pred.append(point_predictions)

plt.scatter([i[1] for i in pred_x], pred, label='unadjusted', c='red', alpha=0.5)

# adjusted predictions

pred = []

for i in pred_x:
    point_predictions = full_model.predict([i for run in range(0, runs)])[...,1] # faster than np.tile for small arrays
    point_predictions = np.array(point_predictions)
    variance = point_predictions.var(axis = 0, ddof = 1) # unbiased estimator
    mean = point_predictions.mean()
    pred.append(mean - 2 * (variance**0.5))

#plt.scatter([i[1] for i in pred_x], pred, label='predictions', c='orange')
plt.scatter([i[1] for i in pred_x], pred, label='adjusted', c='orange', alpha=0.5)

plt.tight_layout()
plt.legend()
plt.savefig('uncertainty_adjustment.png')

In [ ]:
# plot speed vs angle given mu, threshold
mu = 0.05 # set mu
thresh = 4 # set threshold
runs = 10 # runs per prediction

# bug: mu = 0.009 is read as 0.0090..01
#plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
plot_x = [i for i in x if np.isclose(i[0], mu) and i[2] == thresh] # x, y
# this is not generic enough...
pred_x = [[mu,angle,thresh] for angle in np.linspace(0,165,165)]

unadj_pred = []

for i in pred_x:
    point_predictions = full_model.predict([i for run in range(0, 1)])[...,1] # faster than np.tile for small arrays
    point_predictions = np.array(point_predictions)
    unadj_pred.append(point_predictions)
    
adj_pred = []

for i in pred_x:
    point_predictions = full_model.predict([i for run in range(0, runs)])[...,1] # faster than np.tile for small arrays
    point_predictions = np.array(point_predictions)
    variance = point_predictions.var(axis = 0, ddof = 1) # unbiased estimator
    mean = point_predictions.mean()
    adj_pred.append(mean - 2 * (variance**0.5))

In [ ]:
for i,v in enumerate(pred_x):
    v.append(unadj_pred[i][0])
    v.append(adj_pred[i])

In [ ]:
pd.DataFrame(pred_x).to_csv("adjusted_predictions.csv", header=None, index=None)